# Projeto Final
## Análise de Texto de Fontes Desestruturadas e Web.

#### Integrantes
- Manuel Castanares
- Rafael Malcervelli 

### Objetivo

O projeto envolverá o uso das técnicas aprendidas em aula para conduzir, com autonomia, a criação de uma prova de conceito envolvendo a extração de dados de fontes desestruturadas e sua posterior análise.

### Introdução

No mercado de PC gamers, a escolha de componentes é uma decisão crítica que pode afetar significativamente o desempenho e a satisfação do usuário final. Dada a constante flutuação dos preços e a vasta gama de produtos disponíveis, consumidores e montadores de PCs enfrentam o desafio de identificar as melhores opções de compra. Este projeto propõe a utilização de técnicas de web scraping e análise de texto para extrair e analisar dados de múltiplos marketplaces, proporcionando uma ferramenta útil para a comparação de preços, disponibilidade de estoque e qualidade percebida dos componentes.

<p align=center>
    <img src=" ./imgs/pc_gamer_1.jpg" alt="PC GAMER" class="bg-primary mb-1" width="500px">
</p>

### Problema

A principal problemática abordada neste projeto é a dificuldade em obter uma visão clara e atualizada das melhores opções de compra para componentes de PCs gamers devido à volatilidade dos preços e à diversidade de produtos. Esta situação leva a decisões de compra que nem sempre são ótimas em termos de custo-benefício. Portanto, a necessidade de uma solução que agilize e refine o processo de escolha se faz essencial para consumidores e profissionais da área.

A situacao-problema que propomos aqui é a seguinte:<br><br>
Um jovem gostaria de montar um PC gamer que consiga rodar com facilidade os jogos atuais (gráfico avancados, demandando processamento de video) e jogos competitivos. Para isso, esse jovem ja tem em mente a maquina que gostaria de montar com todos os seus componentes e explicacoes dos mesmos, que seguem:

- Placa-mae: O 'coração' do computador, conecta todos os componentes, permitindo que eles se comuniquem. A escolha da placa-mãe influencia diretamente quais processadores e tipos de memória serão compatíveis. -> Placa escolhida para análise (**Asus TUF GAMING Z690-PLUS D4 ATX LGA 1700**). <br><br>
- CPU (Processador): O 'cérebro' do PC, crucial para o processamento de todas as tarefas. O desempenho do processador afeta diretamente a capacidade do sistema em executar jogos complexos sem travamentos. -> CPU escolhida para análise (**Intel Core i7 12700K 3.6 GHz 12-Core**).<br><br>
- GPU (Placa de vídeo): Fundamental para renderizar gráficos avançados em tempo real, a escolha da placa de vídeo é crucial para a experiência visual em jogos. -> GPU escolhida para análise (**Gigabyte GeForce RTX 4060 8 GB Gaming**).<br><br>
- Memória RAM: Responsável por armazenar os dados temporários do sistema, uma quantidade suficiente de RAM é vital para garantir uma operação suave e eficiente durante jogos intensivos. -> Memória escolhida para análise (**Kingston HyperX Fury Black Series (Preto) 8 GB (1x8 GB) DDR4-2666**).<br><br>
- Armazenamento (SSD): Discos de estado sólido (SSD) proporcionam tempos de carregamento mais rápidos para sistemas operacionais e jogos quando comparados aos discos rígidos tradicionais (HDD). -> SSD escolhido para análise (**SanDisk SSD Plus 1 TB 2.5"**).<br><br>
- Fonte de Alimentação: Supre a energia necessária para todos os componentes. Uma fonte de qualidade e com potência adequada é essencial para a estabilidade e longevidade do sistema. -> Fonte escolhida para análise (**Corsair CV550 550 W Certificado 80+ Bronze ATX**).<br><br>
- Fans (Ventoinhas): Cruciais para manter uma temperatura operacional adequada dentro do gabinete, evitando o superaquecimento dos componentes. -> Fan escolhido para análise (**Cooler Master Masterfan Mf120 Halo RGB**).<br><br>
- Cooler para CPU: Específico para o resfriamento do processador, é vital para manter o desempenho sem riscos de superaquecimento, especialmente em sessões prolongadas de jogos. -> Cooler escolhido para análise (**Deepcool AG400 BK**).<br><br>
- Gabinete: Estrutura do PC que irá segurar todas as pecas nos seus devidos lugares. Além da questao estetica, é sempre bom visar por gabinetes que apresentam um bom fluxo de ar dentro deles, permitindo uma maior circulacao e consequentemente, uma diminuicao na temperatura das pecas. -> Gabinete escolhido para análise (** **).

<p align=center>
    <img src=" ./imgs/pc_gamer_2.jpg" alt="PC GAMER 2" class="bg-primary mb-1" width="500px">
</p>

### Extração de dados

Para iniciar a extração, iremos considerar alguns site que apresentam tais produtos para que possamos analisá-los.

### Placa-mae

##### KABUM

In [1]:
import requests
from bs4 import BeautifulSoup

In [45]:
data = []

def get_most_popular_motherboards():
   
    url = "https://www.kabum.com.br/hardware/placas-mae"
    response = requests.get(url)

    
    if response.status_code == 200:
        
        soup = BeautifulSoup(response.content, 'html.parser')

        
        motherboard_elements = soup.find_all('span', class_='sc-d79c9c3f-0 nlmfp sc-9d1f1537-16 fQnige nameCard')
       
        motherboards = [mb.get_text(strip=True) for mb in motherboard_elements]

        
        return motherboards[:10]
    else:
        print("Failed to retrieve motherboard list")
        return []


def get_product_price(product_url):
   
    response = requests.get(product_url)

   
    if response.status_code == 200:
        
        soup = BeautifulSoup(response.content, 'html.parser')

       
        price_element = soup.find('h4', class_='sc-5492faee-2 ipHrwP finalPrice')

       
        if price_element:
            price_text = price_element.get_text(strip=True)
            return price_text
        else:
            return "Price not found"

    else:
        return "Failed to retrieve page"



motherboards = get_most_popular_motherboards()


for mb_name in motherboards:
    
    search_query = mb_name.replace(',', '')
    search_query = search_query.replace(' ', '-')
    search_query = search_query.replace('ã', '%C3%A3')
    search_url = f"https://www.kabum.com.br/busca/{search_query}"
    response = requests.get(search_url, stream=True)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    
    product_url = soup.find('a', class_='sc-9d1f1537-10 kueyFw productLink')['href']
    product_url = 'https://www.kabum.com.br/' + product_url
    
    product_price = get_product_price(product_url)
    product_price = product_price.replace(' ', '')
    product_price = product_price.replace('R$', '')
    product_price = product_price.replace('.', '')
    product_price = product_price.replace(',', '.')
    line_data = [mb_name, 0, float(product_price)]
    data.append(line_data)
print(data)

369.99
784.99
549.99
499.99
359.99
648.99
899.99
474.99
598.99
474.99
[['Placa Mãe MSI A520M-A PRO, AMD AM4, mATX, DDR4', 0, 369.99], ['Placa Mãe Gigabyte B550M Aorus Elite, AMD AM4, Micro ATX, DDR4', 0, 784.99], ['Placa Mãe Asus TUF GAMING A520M-PLUS II, AMD AM4, mATX, DDR4', 0, 549.99], ['Placa Mãe MSI B560M PRO-E, Intel LGA 1200, mATX, DDR4', 0, 499.99], ['Placa Mãe ASRock B450M-HDV R4.0, AMD AM4, Micro ATX, DDR4', 0, 359.99], ['Placa Mãe ASRock B450M Steel Legend, AMD AM4, mATX, DDR4', 0, 648.99], ['Placa-Mãe MSI MPG B550 Gaming Plus, AMD AM4, ATX', 0, 899.99], ['Placa Mãe Gigabyte B450M DS3H V2, AMD B450, mATX, DDR4, (rev. 1.0)', 0, 474.99], ['Placa-Mãe Asus TUF Gaming B450M-Plus II, AMD AM4, mATX, DDR4', 0, 598.99], ['Placa Mãe Gigabyte H610M (rev. 1.0), Intel LGA1700, H610, DDR4, mATX - H610M H DDR4', 0, 474.99]]


In [46]:
def get_most_popular_cpus():
    
    url = "https://www.kabum.com.br/hardware/processadores"
    response = requests.get(url)

    
    if response.status_code == 200:
       
        soup = BeautifulSoup(response.content, 'html.parser')

        
        cpu_elements = soup.find_all('span', class_='sc-d79c9c3f-0 nlmfp sc-9d1f1537-16 fQnige nameCard')
        
        cpus = [mb.get_text(strip=True) for mb in cpu_elements]

        
        return cpus[:10]
    else:
        print("Failed to retrieve cpu list")
        return []


cpus = get_most_popular_cpus()

for cpu_name in cpus:
    
    search_query = cpu_name.replace(',', '')
    search_query = search_query.replace(' ', '-')
    search_query = search_query.replace('ã', '%C3%A3')
    search_url = f"https://www.kabum.com.br/busca/{search_query}"
    response = requests.get(search_url, stream=True)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
   
    product_url = soup.find('a', class_='sc-9d1f1537-10 kueyFw productLink')['href']
    product_url = 'https://www.kabum.com.br/' + product_url
    
    product_price = get_product_price(product_url)
    product_price = product_price.replace(' ', '')
    product_price = product_price.replace('R$', '')
    product_price = product_price.replace('.', '')
    product_price = product_price.replace(',', '.')
    line_data = [cpu_name, 1, float(product_price)]
    data.append(line_data)


In [48]:
def get_most_popular_gpus():
    
    url = "https://www.kabum.com.br/hardware/placa-de-video-vga"
    response = requests.get(url)

    
    if response.status_code == 200:
        
        soup = BeautifulSoup(response.content, 'html.parser')

       
        gpu_elements = soup.find_all('span', class_='sc-d79c9c3f-0 nlmfp sc-9d1f1537-16 fQnige nameCard')
        
        gpus = [mb.get_text(strip=True) for mb in gpu_elements]

       
        return gpus[:10]
    else:
        print("Failed to retrieve gpu list")
        return []


gpus = get_most_popular_gpus()

for gpu_name in gpus:
   
    search_query = gpu_name.replace(',', '')
    search_query = search_query.replace(' ', '-')
    search_query = search_query.replace('ã', '%C3%A3')
    search_url = f"https://www.kabum.com.br/busca/{search_query}"
    response = requests.get(search_url, stream=True)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    
    product_url = soup.find('a', class_='sc-9d1f1537-10 kueyFw productLink')['href']
    product_url = 'https://www.kabum.com.br/' + product_url
    
    product_price = get_product_price(product_url)
    product_price = product_price.replace(' ', '')
    product_price = product_price.replace('R$', '')
    product_price = product_price.replace('.', '')
    product_price = product_price.replace(',', '.')
    line_data = [gpu_name, 2, float(product_price)]
    data.append(line_data)


In [49]:
print(data)

[['Placa Mãe MSI A520M-A PRO, AMD AM4, mATX, DDR4', 0, 369.99], ['Placa Mãe Gigabyte B550M Aorus Elite, AMD AM4, Micro ATX, DDR4', 0, 784.99], ['Placa Mãe Asus TUF GAMING A520M-PLUS II, AMD AM4, mATX, DDR4', 0, 549.99], ['Placa Mãe MSI B560M PRO-E, Intel LGA 1200, mATX, DDR4', 0, 499.99], ['Placa Mãe ASRock B450M-HDV R4.0, AMD AM4, Micro ATX, DDR4', 0, 359.99], ['Placa Mãe ASRock B450M Steel Legend, AMD AM4, mATX, DDR4', 0, 648.99], ['Placa-Mãe MSI MPG B550 Gaming Plus, AMD AM4, ATX', 0, 899.99], ['Placa Mãe Gigabyte B450M DS3H V2, AMD B450, mATX, DDR4, (rev. 1.0)', 0, 474.99], ['Placa-Mãe Asus TUF Gaming B450M-Plus II, AMD AM4, mATX, DDR4', 0, 598.99], ['Placa Mãe Gigabyte H610M (rev. 1.0), Intel LGA1700, H610, DDR4, mATX - H610M H DDR4', 0, 474.99], ['Processador AMD Ryzen 5 4600G, 3.7GHz (4.2GHz Max Turbo), Cache 11MB, AM4, Vídeo Integrado - 100-100000147BOX', 1, 599.99], ['Processador AMD Ryzen 7 5700X3D, 3.6 GHz, (4.1GHz Max Turbo), Cachê 4MB, 8 Núcleos, 16 Threads, AM4 - 100-1000

In [50]:
import pandas as pd

In [51]:
columns = ['Name of product', 'Product type', 'Price (R$)']


df = pd.DataFrame(data, columns=columns)


print(df)

                                      Name of product  Product type  \
0      Placa Mãe MSI A520M-A PRO, AMD AM4, mATX, DDR4             0   
1   Placa Mãe Gigabyte B550M Aorus Elite, AMD AM4,...             0   
2   Placa Mãe Asus TUF GAMING A520M-PLUS II, AMD A...             0   
3   Placa Mãe MSI B560M PRO-E, Intel LGA 1200, mAT...             0   
4   Placa Mãe ASRock B450M-HDV R4.0, AMD AM4, Micr...             0   
5   Placa Mãe ASRock B450M Steel Legend, AMD AM4, ...             0   
6    Placa-Mãe MSI MPG B550 Gaming Plus, AMD AM4, ATX             0   
7   Placa Mãe Gigabyte B450M DS3H V2, AMD B450, mA...             0   
8   Placa-Mãe Asus TUF Gaming B450M-Plus II, AMD A...             0   
9   Placa Mãe Gigabyte H610M (rev. 1.0), Intel LGA...             0   
10  Processador AMD Ryzen 5 4600G, 3.7GHz (4.2GHz ...             1   
11  Processador AMD Ryzen 7 5700X3D, 3.6 GHz, (4.1...             1   
12  Processador AMD Ryzen 5 5600, 3.5GHz (4.4GHz M...             1   
13  Pr

In [52]:
df.head()

,Name of product,Product type,Price (R$)
0,"Placa Mãe MSI A520M-A PRO, AMD AM4, mATX, DDR4",0,369.99
1,"Placa Mãe Gigabyte B550M Aorus Elite, AMD AM4,...",0,784.99
2,"Placa Mãe Asus TUF GAMING A520M-PLUS II, AMD A...",0,549.99
3,"Placa Mãe MSI B560M PRO-E, Intel LGA 1200, mAT...",0,499.99
4,"Placa Mãe ASRock B450M-HDV R4.0, AMD AM4, Micr...",0,359.99


In [53]:
df.to_csv('data.csv', index=False) 